In [1]:
import pandas as pd
from shapely.geometry import Point, Polygon
import geopandas as gpd
fname = '/home/neil/Downloads/NYC/nyc_city_sharad.geojson'
geo_df = gpd.read_file(fname)

In [2]:
data = pd.read_csv('/home/neil/Downloads/NYC/New York Coronavirus Disease Prevention Map Feb 27 2020 Id  Movement between Tiles_2020-02-27 0000.csv')

In [3]:
tmp = data.geometry.str.split(expand=True)

In [4]:
data = data[['date_time', 'length_km', 'tile_size', 'n_crisis', 'n_baseline']]

In [5]:
data

,date_time,length_km,tile_size,n_crisis,n_baseline
0,2020-02-27 0000,1.853778,14,12,14.8
1,2020-02-27 0000,1.850767,14,66,60.4
2,2020-02-27 0000,2.617052,14,17,16.2
3,2020-02-27 0000,4.137404,14,17,13.4
4,2020-02-27 0000,5.850440,14,12,14.0
...,...,...,...,...,...
6240,2020-02-27 0000,1.850535,14,147,136.6
6241,2020-02-27 0000,5.550215,14,10,14.2
6242,2020-02-27 0000,9.248040,14,17,13.6
6243,2020-02-27 0000,7.399359,14,13,12.8


In [6]:
data[['type', 'long_start', 'lat_start', 'long_end', 'lat_end']] = tmp

In [7]:
data = data.drop(columns='type')

In [9]:
data

,date_time,length_km,tile_size,n_crisis,n_baseline,long_start,lat_start,long_end,lat_end
0,2020-02-27 0000,1.853778,14,12,14.8,(-73.575439453125,"40.63896682325716,",-73.575439453125,40.65563821945469)
1,2020-02-27 0000,1.850767,14,66,60.4,(-73.92700195312499,"40.755579121875044,",-73.948974609375,40.755579121875044)
2,2020-02-27 0000,2.617052,14,17,16.2,(-73.92700195312499,"40.755579121875044,",-73.948974609375,40.772221356403094)
3,2020-02-27 0000,4.137404,14,17,13.4,(-73.92700195312499,"40.755579121875044,",-73.948974609375,40.78885942352399)
4,2020-02-27 0000,5.850440,14,12,14.0,(-73.92700195312499,"40.755579121875044,",-73.948974609375,40.80549332287863)
...,...,...,...,...,...,...,...,...,...
6240,2020-02-27 0000,1.850535,14,147,136.6,(-73.92700195312499,"40.755579121875044,",-73.927001953125,40.772221356403094)
6241,2020-02-27 0000,5.550215,14,10,14.2,(-73.92700195312499,"40.755579121875044,",-73.927001953125,40.80549332287863)
6242,2020-02-27 0000,9.248040,14,17,13.6,(-73.92700195312499,"40.755579121875044,",-73.927001953125,40.838748616860215)
6243,2020-02-27 0000,7.399359,14,13,12.8,(-73.92700195312499,"40.755579121875044,",-73.92700195312501,40.82212305410941)


In [10]:
data['long_start'] = [float(i) for i in data['long_start'].str.lstrip('(')]
data['lat_start'] = [float(i) for i in data['lat_start'].str.rstrip(',')]
data['lat_end'] = [float(i) for i in data['lat_end'].str.rstrip(')')]
data['long_end'] = [float(i) for i in data['long_end'].values]

In [12]:
data.to_csv('/home/neil/finaldata.csv')

In [ ]:
#gdf1 = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy( data['long_start'], data['lat_start']))
gdf2 = gpd.GeoDataFrame(data, geometry=gpd.points_from_xy( data['long_end'], data['lat_end']))

In [ ]:
gdf2

In [ ]:
#data['p1'] = gdf1['geometry']
data['p2'] = gdf2['geometry']

In [ ]:
data = data.drop(columns='geometry')

In [ ]:
data

In [ ]:
# wards1 = []
# for i in data['p1'].values:
#     for j in geo_df[['geometry', 'wardNo']].values:
#         if i.within(j[0]):
#             wards1.append([i, j[1]])
wards2 = []
for i in data['p2'].values:
    for j in geo_df[['geometry', 'wardNo']].values:
        if i.within(j[0]):
            wards2.append([i, j[1]])
        

In [ ]:
# start_ward_df = data[data.p1.isin([i[0] for i in wards1])]
# start_ward_df['start_ward']=[i[1] for i in wards1]
end_ward_df = data[data.p2.isin([i[0] for i in wards2])]
end_ward_df['end_ward']=[i[1] for i in wards2]

In [ ]:
end_ward_df.to_csv('ward2.csv')

In [ ]:
end_ward_df

In [13]:
start_ward_df = pd.read_csv('/home/neil/ward1.csv')
start_ward_df = start_ward_df.drop(columns=['long_start','lat_start','long_end','lat_end','p1'])

In [14]:
end_ward_df = pd.read_csv('/home/neil/ward2.csv')
end_ward_df= end_ward_df.drop(columns=['long_start','lat_start','long_end','lat_end','p2'])

In [15]:
end_ward_df = end_ward_df.rename(columns={'Unnamed: 0': "num"})
start_ward_df = start_ward_df.rename(columns={'Unnamed: 0': "num"})

In [16]:
end_ward_df

,num,date_time,length_km,tile_size,n_crisis,n_baseline,end_ward
0,1,2020-02-27 0000,1.850767,14,66,60.4,170
1,2,2020-02-27 0000,2.617052,14,17,16.2,111
2,3,2020-02-27 0000,4.137404,14,17,13.4,112
3,4,2020-02-27 0000,5.850440,14,12,14.0,95
4,6,2020-02-27 0000,2.617707,14,13,12.8,142
...,...,...,...,...,...,...,...
4751,6240,2020-02-27 0000,1.850535,14,147,136.6,172
4752,6241,2020-02-27 0000,5.550215,14,10,14.2,24
4753,6242,2020-02-27 0000,9.248040,14,17,13.6,13
4754,6243,2020-02-27 0000,7.399359,14,13,12.8,35


In [17]:
start_ward_df

,num,date_time,length_km,tile_size,n_crisis,n_baseline,start_ward
0,1,2020-02-27 0000,1.850767,14,66,60.4,171
1,2,2020-02-27 0000,2.617052,14,17,16.2,171
2,3,2020-02-27 0000,4.137404,14,17,13.4,171
3,4,2020-02-27 0000,5.850440,14,12,14.0,171
4,5,2020-02-27 0000,9.431188,14,16,11.2,171
...,...,...,...,...,...,...,...
4799,6240,2020-02-27 0000,1.850535,14,147,136.6,171
4800,6241,2020-02-27 0000,5.550215,14,10,14.2,171
4801,6242,2020-02-27 0000,9.248040,14,17,13.6,171
4802,6243,2020-02-27 0000,7.399359,14,13,12.8,171


In [18]:
final_data = pd.merge(start_ward_df, end_ward_df[['num', 'end_ward']], on="num")

In [19]:
final_data.groupby(['start_ward', 'end_ward'])['length_km'].count().unstack().to_csv('/home/neil/distance_matrix_count.csv')

In [20]:
final_data

,num,date_time,length_km,tile_size,n_crisis,n_baseline,start_ward,end_ward
0,1,2020-02-27 0000,1.850767,14,66,60.4,171,170
1,2,2020-02-27 0000,2.617052,14,17,16.2,171,111
2,3,2020-02-27 0000,4.137404,14,17,13.4,171,112
3,4,2020-02-27 0000,5.850440,14,12,14.0,171,95
4,6,2020-02-27 0000,2.617707,14,13,12.8,171,142
...,...,...,...,...,...,...,...,...
4362,6240,2020-02-27 0000,1.850535,14,147,136.6,171,172
4363,6241,2020-02-27 0000,5.550215,14,10,14.2,171,24
4364,6242,2020-02-27 0000,9.248040,14,17,13.6,171,13
4365,6243,2020-02-27 0000,7.399359,14,13,12.8,171,35


In [21]:
final_data.groupby(['start_ward', 'end_ward'])['length_km'].sum().unstack().to_csv('/home/neil/distance_matrix_sum.csv')

In [22]:
final_data.groupby(['start_ward', 'end_ward'])['length_km'].max().unstack().to_csv('/home/neil/distance_matrix.csv')